In [4]:
from __future__ import print_function 
import os, sys, h5py
import numpy as np
from six.moves import cPickle
from collections import OrderedDict

import tensorflow as tf

sys.path.append('..')
from tfomics import layers, utils, init, learn, explore
from tfomics import neuralnetwork as nn
from tfomics.build_network import *

# import models
from model_zoo import simple_connectomics_model_explore

In [5]:
filename = 'processed_dataset.hdf5'
data_path = '/Users/juliankimura/Desktop/tensorflow/data'
filepath = os.path.join(data_path,filename)

group_name = ['processed_data']
dataset = h5py.File(filepath,'r')
%time dtf = np.array(dataset['/'+group_name[0]+'/dtf'])
ltf = np.array(dataset['/'+group_name[0]+'/ltf'])
dtf_crossval = np.array(dataset['/'+group_name[0]+'/dtf_crossval'])
ltf_crossval = np.array(dataset['/'+group_name[0]+'/ltf_crossval'])

X_train = dtf.transpose([0,2,3,1])
y_train = ltf
X_valid = dtf_crossval.transpose([0,2,3,1])
y_valid = ltf_crossval


CPU times: user 1.3 s, sys: 6.18 s, total: 7.48 s
Wall time: 8.42 s


In [9]:

def model(input_shape, num_labels):

    # placeholders
    inputs = utils.placeholder(shape=input_shape, name='input')
    is_training = tf.placeholder(tf.bool, name='is_training')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    targets = utils.placeholder(shape=(None,num_labels), name='output')

    # placeholder dictionary
    placeholders = {'inputs': inputs, 
                    'targets': targets, 
                    'keep_prob': keep_prob, 
                    'is_training': is_training}

    # create model
    layer1 = {'layer': 'input',
                'inputs': inputs,
              'name': 'input'
                }
    layer2 = {'layer': 'conv1d', 
                'num_filters': {'start': 25, 'bounds': [1, 200], 'scale': 20},
                'filter_size': {'start': 9, 'bounds': [3, 27], 'scale': 10, 'multiples': 2, 'offset': 1},
                'norm': 'batch',
                'padding': 'SAME',
                'activation': 'relu',
                'pool_size': {'start': 2, 'bounds': [1, 20], 'scale': 6, 'multiples': 2},
                'name': 'conv1'
                }
    layer3 = {'layer': 'conv1d', 
                'num_filters': {'start': 50, 'bounds': [1, 200], 'scale': 30},
                'filter_size': {'start': 7, 'bounds': [3, 23], 'scale': 10, 'multiples': 2, 'offset': 1},
                'norm': 'batch',
                'padding': 'SAME',
                'activation': 'relu',
                'pool_size': {'start': 2, 'bounds': [1, 10], 'scale': 4, 'multiples': 2},
                'name': 'conv1'
                }
    layer4 = {'layer': 'dense', 
                'num_units': {'start': 64, 'bounds': [16, 1000], 'scale': 50},
                'norm': 'batch',
                'activation': 'relu',
                'name': 'dense1'
                }
    layer5 = {'layer': 'dense', 
              'num_units': num_labels,
              'activation': 'sigmoid',
              'name': 'dense2'
                }

    #from tfomics import build_network
    model_layers = [layer1, layer2, layer3, layer4, layer5]

    # optimization parameters
    optimization = {"objective": "binary",
                    "optimizer": "adam",
                    "learning_rate": 0.001,
                    "l2": 1e-6,
                    #"learning_rate": {'start': -3, 'bounds': [-4, -1], 'scale': 1.5, 'transform': 'log'},      
                    #"l2": {'start': -6, 'bounds': [-8, -2], 'scale': 3, 'transform': 'log'},
                    # "l1": 0, 
                    }
    return model_layers, placeholders, optimization


# build network
input_shape = list(X_train.shape)
input_shape[0] = None
num_labels = y_train.shape[1]
model_layers, placeholders, optimization = model(input_shape, num_labels)

# build neural opti
optimizer = explore.NeuralOptimizer(model_layers, placeholders, optimization)


In [10]:
optimizer.sample_network()

[{'inputs': <tf.Tensor 'input_1:0' shape=(?, 330, 1, 3) dtype=float32>,
  'layer': 'input',
  'name': 'input'},
 {'activation': 'relu',
  'filter_size': 6,
  'layer': 'conv1d',
  'name': 'conv1',
  'norm': 'batch',
  'num_filters': 36,
  'padding': 'SAME',
  'pool_size': 16},
 {'activation': 'relu',
  'filter_size': 18,
  'layer': 'conv1d',
  'name': 'conv1',
  'norm': 'batch',
  'num_filters': 18,
  'padding': 'SAME',
  'pool_size': 2},
 {'activation': 'relu',
  'layer': 'dense',
  'name': 'dense1',
  'norm': 'batch',
  'num_units': 86},
 {'activation': 'sigmoid', 'layer': 'dense', 'name': 'dense2', 'num_units': 2}]

In [ ]:

# optimize model parameters
train = {'inputs': X_train, 'targets': y_train, 'keep_prob': 0.8, 'is_training': True}
valid = {'inputs': X_valid, 'targets': y_valid, 'keep_prob': 1, 'is_training': False}        
optimizer.optimize(train, valid, num_trials=20, num_epochs=5, batch_size=128, verbose=0)


---------------------------------------------------------
Running baseline model

Model layers:
name: input
pool_size: 2
name: conv1
num_filters: 25
filter_size: 9
pool_size: 2
name: conv1
num_filters: 50
filter_size: 7
name: dense1
num_units: 64
name: dense2
num_units: 2

Optimization:
learning_rate: 0.001
l2: 1e-06

Epoch 1 out of 5 


In [ ]:
# save optimal model
optimizer.print_optimal_model()